In [ ]:
# 导入包(PypI)
import re
import time
import random
import pymongo
import pandas as pd

from datetime import datetime
from tqdm import tqdm

from scrapy.selector import Selector

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **cmd** 命令提示符<输入以下命令（将 `your Chrome.exe path` 替换为您的 Chrome 浏览器路径）：
```bash
& "your Chrome.exe path" --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)

In [ ]:
# 数据清洗可能需要使用的方法
def extract_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue
        
        number = re.findall(r'\d+\.?\d*', x)[0]
        if number:
            list_cleaned.append(number)
        else:
            list_cleaned.append(None)
    
    return list_cleaned

def extract_large_number(List):
    list_cleaned = []
    for x in List:
        if x is None or not isinstance(x, str):
            list_cleaned.append(None)
            continue
        
        if '万' in x:
            number = x.replace('万', '')
            number = float(number) * 10000
            number = int(number)
        else:
            number = int(float(x))
        
        list_cleaned.append(number)
        
    return list_cleaned

def extract_date(List):
    list_cleaned = []
    for x in List:
        match = re.search(r'(\d{4})-(\d{2})-(\d{2})', x)
        if match:
            date_cleand = match.group(0)
        else:
            match = re.search(r'(\d{2})-(\d{2})', x)
            if match:
                current_year = datetime.now().year
                month, day = match.groups()
                date_cleand = f'{current_year}-{month}-{day}'
        
        list_cleaned.append(date_cleand)
    
    return list_cleaned

In [ ]:
# 配置Chrome浏览器
service = Service("C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe")

options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')# 远程调控模式启用
options.add_argument('--incognito')# 隐身/无痕模式启用
browser = webdriver.Chrome(service=service, options=options)
action = ActionChains(browser)

In [ ]:
key_word = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_page_load_status(browser, keyword):
    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    
    while True:
        if keyword in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(2)

def selenium_test():
    """
    登录状态检查，网页加载检查，根据用户输入进行搜索
    """
    global key_word, num
    browser.get('https://www.xiaohongshu.com/explore')
    
    check_login_status(browser)
    
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    keyword = input("搜索关键词：")
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    url = f'https://www.xiaohongshu.com/search_result?keyword={keyword}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)

    check_page_load_status(browser, keyword)

selenium_test()

In [ ]:
def change_mode(browser):
    # 更改模式为图文
    try:
        mode_button = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
        mode_button.click()
        print('已自动更改模式为图文。')
    except Exception as e:
        print(f"更改模式失败: {e}")

selected_order_text = ''
def change_sort_order(browser, action):
    # 更改排序方式
    sort_order = {
        '综合': 1,
        '最新': 2,
        '最热': 3
    }
    print("请选择排序方式:")
    for idx, order in sort_order.items():
        print(f'{order}. {idx}')
    
    try:
        global selected_order_text
        selected_order_text = input("请输入排序方式对应的名称: ").strip()
        if selected_order_text not in sort_order:
            print("请输入有效的排序方式...")
            return
        
        selected_order_index = sort_order[selected_order_text]
    except Exception as e:
        print(f"处理排序选择时出错: {e}")
        return

    try:
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()# 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()# 模拟鼠标点击

        print('已选择排序方式为:',selected_order_text)
        print('检查时间:',time.ctime())

    except Exception as e:
        print(f"更改排序方式失败: {e}")

change_mode(browser)
change_sort_order(browser, action)

In [ ]:
def parsePage(html_content, authorName_list, likeNr_list, URL_list, userURL_list, num):
    """
    解析网页内容并更新数据列表。

    Args:
        html_content (str): 当前页面的HTML内容
        authorName_list (list): 存储作者名字的列表
        likeNr_list (list): 存储获赞数量的列表
        URL_list (list): 存储笔记URL的列表
        userURL_list (list): 存储用户URL的列表
        qbar (tqdm): 进度条对象
        num (int): 需要爬取的笔记数量

    Returns:
        None: 数据存储在传入的列表中
    """
    response = Selector(text=html_content)
    divs = response.xpath('//div[contains(@class, "feeds-container")]/section/div')# 选中网页中包含笔记信息的部分

    # 遍历divs获取每一篇笔记的信息
    for div in divs:
        if len(URL_list) >= num:
            break
        
        if div.xpath('.//span[contains(text(), "大家都在搜")]'):
            continue

        # 选择并提取网页数据
        try:
            author_name = div.xpath('.//a[contains(@class, "author")]/span[contains(@class, "name")]/text()').get()# 作者名字
            like_nr = div.xpath('.//span[contains(@class, "count")]/text()').get()# 获赞数量
            url = div.xpath('.//a[contains(@class, "cover")]/@href').get()# 笔记URL
            user_url = div.xpath('.//a[contains(@class, "author")]/@href').get()# 用户URL
            
            authorName_list.append(author_name)
            likeNr_list.append(like_nr)
            URL_list.append(url)
            userURL_list.append(user_url)

            time.sleep(0.35)
            
        except:
            pass
    
    return True

authorName_list, likeNr_list, URL_list, userURL_list = [], [], [], []
qbar = tqdm(total=num, desc="已获取的笔记数量...")

# 检查是否已经爬取足够数量的笔记，或是否已经达到页面底部
while len(URL_list) < num:
    if '- THE END -' in browser.page_source:
        print(f"当前与{key_word}有关的笔记数量少于 {num}")
        print('检查时间:',time.ctime())
        break
    
    parsePage(browser.page_source, authorName_list, likeNr_list, URL_list, userURL_list, num)
    qbar.update(1)

    if len(URL_list) < num:
        browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')# 模拟鼠标滚动
        time.sleep(random.uniform(3, 5))

if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]

qbar.close()

In [ ]:
if all(len(lst) == num for lst in [authorName_list, likeNr_list, URL_list, userURL_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [authorName_list, likeNr_list, URL_list, userURL_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

In [ ]:
likeNr_list = extract_large_number(likeNr_list)
URL_list = [re.sub(r'^/search_result/', '/', url) for url in URL_list]
userURL_list = [url.split('/')[-1] for url in userURL_list]

print("以下为清洁数据示例:\n")
for i in range(3):
    print("author_name:", authorName_list[i])
    print("like_nr:", likeNr_list[i])
    print("url:", URL_list[i])
    print("user_url:", userURL_list[i])
    print("------")

In [ ]:
def parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list):
    """
    解析单个笔记页面并提取所需数据
    
    Args:
        browser: Selenium WebDriver 实例，用于获取页面内容
        url: 笔记的URL
        commentNr_list (list): 存储评论数量的列表
        content_list (list): 存储笔记内容的列表
        datePublished_list (list): 存储发布时间的列表
        images_list (list): 存储图片链接的列表
        starNr_list (list): 存储收藏数量的列表

    Returns:
        None: 将提取的数据添加到相应的列表中
    """
    whole_url = 'https://www.xiaohongshu.com/explore' + url# 构造完整笔记URL
    browser.get(whole_url)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))# 等待页面加载完成
    html = browser.page_source
    selector = Selector(text = html)
    
    try:
        # 选择并提取网页数据
        comment_nr = selector.xpath('//*[@class="total"]/text()').extract_first()# 评论数量
        content = selector.xpath('//*[@name="description"]/@content').extract_first()# 内容
        datePublished = selector.xpath('//*[@class="date"]/text()').extract_first()# 发布时间
        images = selector.xpath('//*[@name="og:image"]/@content').extract_first()# 图片
        images = images + '.jpg' if images else None# 追加图片链接
        star_nr = selector.xpath('//*[@class="count"]/text()').extract_first()# 收藏数量

        commentNr_list.append(comment_nr)
        content_list.append(content)
        datePublished_list.append(datePublished)
        images_list.append(images)
        starNr_list.append(star_nr)

    except:
        pass
    
commentNr_list, content_list, datePublished_list, images_list, starNr_list = [], [], [], [], []

qbar = tqdm(total=len(URL_list), desc="已获取的笔记数量...")
for url in URL_list:
    parse_note_page(browser, url, commentNr_list, content_list, datePublished_list, images_list, starNr_list)
    qbar.update(1)
    time.sleep(random.uniform(0.5, 2))

qbar.close()

In [ ]:
if all(len(lst) == num for lst in [commentNr_list, content_list, datePublished_list, images_list, starNr_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [commentNr_list, content_list, datePublished_list, images_list, starNr_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

In [ ]:
commentNr_list = extract_large_number(extract_number(commentNr_list))
starNr_list = extract_large_number(starNr_list)
datePublished_list = extract_date(datePublished_list)

print("以下为清洁数据示例:\n")
for i in range(3):
    print("comment_nr:", commentNr_list[i])
    print("content:", content_list[i])
    print("datePublished:", datePublished_list[i])
    print("images:", images_list[i])
    print("star_nr:", starNr_list[i])
    print("------")

In [ ]:
def parse_author_page(browser, user_url, authorCollectNr_list, authorFansNr_list, authorNoteNr_list):
    """
    解析单个用户页面并提取所需数据
    
    Args:
        browser: Selenium WebDriver 实例，用于获取页面内容
        user_url: 用户的URL
        authorCollectNr_list (list): 存储作者获赞与收藏数量的列表
        authorFansNr_list (list): 存储作者粉丝数量的列表
        authorNoteNr_list (list): 存储作者笔记数量的列表
    
    Returns:
        None: 将提取的数据添加到相应的列表中
    """
    whole_url = 'https://www.xiaohongshu.com/user/profile/' + user_url# 构造完整用户页面URL
    browser.get(whole_url)

    # 模拟滚动页面直到加载完成
    while True:
        previous_page_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# 滚动到页面底部
        time.sleep(random.uniform(1, 2))

        current_page_height = browser.execute_script("return document.body.scrollHeight")
        if current_page_height == previous_page_height:
            break
        previous_page_height = current_page_height

    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="userPostedFeeds"]//section')))# 等待页面加载完成
    html = browser.page_source
    selector = Selector(text=html)

    try:
        # 选择并提取网页数据
        author_collect_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[3]/span[@class="count"]/text()').extract_first()# 作者获赞与收藏数量
        author_fans_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()# 作者粉丝数量
        author_note_nr = len(selector.xpath('//*[@id="userPostedFeeds"]//section'))# 作者笔记数量

        authorCollectNr_list.append(author_collect_nr)
        authorFansNr_list.append(author_fans_nr)
        authorNoteNr_list.append(author_note_nr)

    except Exception as e:
        pass

authorCollectNr_list, authorFansNr_list, authorNoteNr_list = [], [], []

qbar = tqdm(total=len(userURL_list), desc="已爬取的作者数量...")
for user_url in userURL_list:
    parse_author_page(browser, user_url, authorCollectNr_list, authorFansNr_list, authorNoteNr_list)
    qbar.update(1)
    time.sleep(random.uniform(1, 2))

qbar.close()

In [ ]:
if all(len(lst) == num for lst in [authorCollectNr_list, authorFansNr_list, authorNoteNr_list]):
    print(f"所有属性列表长度均为 {num}，爬取成功！")
    print(f'检查时间: {time.ctime()}')
else:
    min_length = min(map(len, [authorCollectNr_list, authorFansNr_list, authorNoteNr_list]))
    print(f"当前属性列表长度最小值为 {min_length}，请重新运行上一代码单元，直至所有属性列表长度均为 {num}！")
    print(f'检查时间: {time.ctime()}')

In [ ]:
authorCollectNr_list = extract_large_number(authorCollectNr_list)
authorFansNr_list = extract_large_number(authorFansNr_list)

print("以下为清洁数据示例：")
for i in range(3):
    print("author_collect_nr:", authorCollectNr_list[i])
    print("author_fans_nr:", authorFansNr_list[i])
    print("author_note_nr:", authorNoteNr_list[i])
    print("------")

In [ ]:
dic={
    "author_collect_nr": authorCollectNr_list,# 作者获赞与收藏数量
    "author_fans_nr": authorFansNr_list,# 粉丝数量
    "author_name": authorName_list,# 作者名字
    "author_note_nr": authorNoteNr_list,# 作者笔记数量
    "comment_nr": commentNr_list,# 笔记评论数量
    "content": content_list,# 笔记内容
    "datePublished": datePublished_list,# 笔记发布日期
    "images": images_list,# 笔记封面图片
    "like_nr": likeNr_list,# 笔记获赞数量
    "url": URL_list,# 笔记URL
    "user_url": userURL_list# 作者URL
    }

df = pd.DataFrame.from_dict(dic)
df = df[~df.duplicated(keep='first')]# 检索并删除所有属性值都相同的行,即保留第一次出现的行，删除后续的重复行
print("删除", num-len(df), "行重复行后剩余", len(df), "行。")
print('检查时间:',time.ctime())
df.head(10)

In [ ]:
col_name = f"{key_word}{selected_order_text}笔记Top{num}"

# 连接MongoDB
myclient=pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient["小红书关键词笔记数据"]#进行操作的库名
mycol=mydb[col_name]#进行操作的集合名

mycol.insert_many(df.to_dict('records'))

myclient.close()# 关闭MongoDB客户端连接
browser.close()# 关闭Chrome